In [3]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Sets variables username and password to the correct username and password to access the  DB
USER = "aacuser"
PASS = "SNHU1234"
HOST = 'nv-desktop-services.apporto.com'
PORT = 32325
DB = 'AAC'
COL = 'animals'

# Connect to database via CRUD Module
db = AnimalShelter(USER, PASS, HOST, PORT, DB, COL)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    html.Img(src='data:Image/png;base64,{}'.format(encoded_image.decode())),
    html.H1("Module 7 Project - Mason Williams"), # Unique identifier for course
    
    #create radiobuttons for filtering
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Reset', 'value': 'All'},
            {'label':'Water Rescue', 'value': 'Water Rescue'},
            {'label':'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
            {'label':'Disaster Rescue or Individual Tracking', 'value':'Disaster Rescue or Individual Tracking'},
        ],
        value='All'
    ),
    # Creates the interactive Dash DataTable for displaying animal records
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True}
            for i in df.columns
        ],
        data=df.to_dict('records'),  # Populate table with MongoDB data
        editable=False,  # Disable editing of table data
        filter_action="native",  # Enable filtering
        sort_action="native",  # Enable sorting
        sort_mode="multi",  # Allow multi-column sorting
        column_selectable=False,  # Disable column selection
        row_selectable="single",  # Allow selection of a single row
        row_deletable=False,  # Disable row deletion
        selected_columns=[],  # Initialize selected columns as empty
        selected_rows=[0],  # Default selection to the first row
        page_action="native",  # Enable native pagination
        page_current=0,  # Start on first page
        page_size=10  # Show 10 rows per page
    ),
    html.Br(),
    html.Hr(),
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex', 'justify-content':'center'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])



#############################################
# Interaction Between Components / Controller
#############################################
@app.callback([Output('datatable-id','data'),
              Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    WATER_RESCUE_BREEDS = [
        "Labrador Retriever Mix",
        "Chesapeake Bay Retriever",
        "Newfoundland"
    ]
    MOUNTAIN_WILDERNESS_BREEDS = [
        "German Shepherd",
        "Alaskan Malamute",
        "Old English Sheepdog",
        "Siberian Husky",
        "Rottweiler"
    ]
    DISASTER_TRACKING_BREEDS = [
        "Doberman Pinscher",
        "German Shepherd",
        "Golden Retriever",
        "Bloodhound",
        "Rottweiler"
    ]
    
    # Build query
    if filter_type == "Water Rescue":
        # Water: Intact Female, breed in set, 26 <= age <= 156
        query = {
            "$and": [
                {"breed": {"$in": WATER_RESCUE_BREEDS}},
                {"sex_upon_outcome": "Intact Female"},
                {"age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
            ]
        }
    elif filter_type == "Mountain or Wilderness Rescue":
        # Mountain/Wilderness: Intact Male, breed in set, 26 <= age <= 156
        query = {
            "$and": [
                {"breed": {"$in": MOUNTAIN_WILDERNESS_BREEDS}},
                {"sex_upon_outcome": "Intact Male"},
                {"age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
            ]
        }
    elif filter_type == "Disaster Rescue or Individual Tracking":
        # Disaster/Tracking: Intact Male, breed in set, 20 <= age <= 300
        query = {
            "$and": [
                {"breed": {"$in": DISASTER_TRACKING_BREEDS}},
                {"sex_upon_outcome": "Intact Male"},
                {"age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}}
            ]
        }
    else:
        # "All" -> return all documents
        query = {}

    # Fetch matching documents
    data_cursor = db.read(query)
    
    # Convert to DataFrame
    df = pd.DataFrame(data_cursor)
    
    #remove object id column to rid of errors
    df.drop(columns=["_id"], errors='ignore', inplace=True)

        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
       
       
    return (data,columns)
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    dffPie = pd.DataFrame.from_dict(viewData)

    return [
        dcc.Graph(            
            figure = px.pie(dffPie, names='breed',)
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Connected to DB: AAC and collection: animals
READ CALLED
RESULTS BEING PRINTED
{'_id': ObjectId('67a92f1f636efaba5c0477b2'), 'animal_id': 'A654321', 'animal_type': 'Mongoose', 'breed': 'White Tailed', 'color': 'Brown/Grey'}
{'_id': ObjectId('67a931a221c68fe10e07c7b4'), 'rec_num': 3, 'age_upon_outcome': '2 years', 'animal_id': 'A716330', 'animal_type': 'Dog', 'breed': 'Chihuahua Shorthair Mix', 'color': 'Brown/White', 'date_of_birth': '2013-11-18', 'datetime': '2015-12-28 18:43:00', 'monthyear': '2015-12-28T18:43:00', 'name': 'Frank', 'outcome_subtype': '', 'outcome_type': 'Adoption', 'sex_upon_outcome': 'Neutered Male', 'location_lat': 30.7595748121648, 'location_long': -97.5523753807133, 'age_upon_outcome_in_weeks': 110.111408730159}
{'_id': ObjectId('67a931a221c68fe10e07c7b5'), 'rec_num': 1, 'age_upon_outcome': '3 years', 'animal_id': 'A746874', 'animal_type': 'Cat', 'breed': 'Domestic Shorthair Mix', 'color': 'Black/White', 'date_of_birth': '2014-04-10', 'datetime': '2017-04-11 09:0

READ CALLED
RESULTS BEING PRINTED
{'_id': ObjectId('67a931a221c68fe10e07c7d4'), 'rec_num': 36, 'age_upon_outcome': '6 months', 'animal_id': 'A706953', 'animal_type': 'Dog', 'breed': 'Labrador Retriever Mix', 'color': 'Yellow', 'date_of_birth': '2014-12-06', 'datetime': '2015-07-06 11:33:00', 'monthyear': '2015-07-06T11:33:00', 'name': '', 'outcome_subtype': 'Medical', 'outcome_type': 'Euthanasia', 'sex_upon_outcome': 'Intact Female', 'location_lat': 30.5480802368633, 'location_long': -97.2969969058957, 'age_upon_outcome_in_weeks': 30.3544642857143}
{'_id': ObjectId('67a931a221c68fe10e07ca8e'), 'rec_num': 732, 'age_upon_outcome': '2 years', 'animal_id': 'A749782', 'animal_type': 'Dog', 'breed': 'Labrador Retriever Mix', 'color': 'Tan/White', 'date_of_birth': '2015-05-19', 'datetime': '2017-07-25 14:59:00', 'monthyear': '2017-07-25T14:59:00', 'name': '*Catalina', 'outcome_subtype': '', 'outcome_type': 'Return to Owner', 'sex_upon_outcome': 'Intact Female', 'location_lat': 30.613831063675

READ CALLED
RESULTS BEING PRINTED
{'_id': ObjectId('67a931a221c68fe10e07c7d4'), 'rec_num': 36, 'age_upon_outcome': '6 months', 'animal_id': 'A706953', 'animal_type': 'Dog', 'breed': 'Labrador Retriever Mix', 'color': 'Yellow', 'date_of_birth': '2014-12-06', 'datetime': '2015-07-06 11:33:00', 'monthyear': '2015-07-06T11:33:00', 'name': '', 'outcome_subtype': 'Medical', 'outcome_type': 'Euthanasia', 'sex_upon_outcome': 'Intact Female', 'location_lat': 30.5480802368633, 'location_long': -97.2969969058957, 'age_upon_outcome_in_weeks': 30.3544642857143}
{'_id': ObjectId('67a931a221c68fe10e07ca8e'), 'rec_num': 732, 'age_upon_outcome': '2 years', 'animal_id': 'A749782', 'animal_type': 'Dog', 'breed': 'Labrador Retriever Mix', 'color': 'Tan/White', 'date_of_birth': '2015-05-19', 'datetime': '2017-07-25 14:59:00', 'monthyear': '2017-07-25T14:59:00', 'name': '*Catalina', 'outcome_subtype': '', 'outcome_type': 'Return to Owner', 'sex_upon_outcome': 'Intact Female', 'location_lat': 30.613831063675

READ CALLED
RESULTS BEING PRINTED
{'_id': ObjectId('67a931a221c68fe10e07c7d4'), 'rec_num': 36, 'age_upon_outcome': '6 months', 'animal_id': 'A706953', 'animal_type': 'Dog', 'breed': 'Labrador Retriever Mix', 'color': 'Yellow', 'date_of_birth': '2014-12-06', 'datetime': '2015-07-06 11:33:00', 'monthyear': '2015-07-06T11:33:00', 'name': '', 'outcome_subtype': 'Medical', 'outcome_type': 'Euthanasia', 'sex_upon_outcome': 'Intact Female', 'location_lat': 30.5480802368633, 'location_long': -97.2969969058957, 'age_upon_outcome_in_weeks': 30.3544642857143}
{'_id': ObjectId('67a931a221c68fe10e07ca8e'), 'rec_num': 732, 'age_upon_outcome': '2 years', 'animal_id': 'A749782', 'animal_type': 'Dog', 'breed': 'Labrador Retriever Mix', 'color': 'Tan/White', 'date_of_birth': '2015-05-19', 'datetime': '2017-07-25 14:59:00', 'monthyear': '2017-07-25T14:59:00', 'name': '*Catalina', 'outcome_subtype': '', 'outcome_type': 'Return to Owner', 'sex_upon_outcome': 'Intact Female', 'location_lat': 30.613831063675